In [2]:
import io, os, sys, types
import graphlab as gl
from datetime import datetime
from os import path
import numpy as np
import import_ipynb
from decimal import *
import Conn_E_SQL as Conn
import Creacion_Vectores as C_V
import Normalizacion as Norm
from collections import OrderedDict
import operator

#Atributos de BD
host="recomdb.c4v5c6aergun.us-west-2.rds.amazonaws.com"
user="imaginex"
passwd="6^b{X2Du)Qf6"
database="RecomDB"
table= "CBF"

#Atributos de XML o consola

print ("Quieres ver apartamentos fuera de tu zona de escogencia?")
visualization=int(input())
Type=2

#Atributos de Vivienda

Vector_BD_Vivienda= Conn.Conexion_BD(host,user,passwd,database,table)
apartment_file_processed= gl.SFrame(C_V.Creacion_Vector_Apartamento(Vector_BD_Vivienda))
apartment_file_normalized=gl.SFrame(Norm.norm_apartment_matrix(apartment_file_processed))
Vector_IDF=[]
Vector_IDF=Norm.Inv_DF(apartment_file_processed)

#Atributos de Clientes

clientfile="C:\\Users\\user\\Documents\\Sistema de Recomendacion\\Single_Client_raw.csv"
client_file_processed=gl.SFrame(C_V.Creacion_Vector_Cliente(clientfile))
client_array=client_file_processed["ID_C"]
client_number=client_array[0]
zona_array=client_file_processed["Zona"]
zona_number=zona_array[0]

#Atributos de Rating

Vector_Ratings_BD=Conn.Conexion_BD_rating(client_number, host, user, passwd, database, table)
client_rating=C_V.Creacion_Matrix_Rating(Vector_Ratings_BD)

#Atributos de Ratings

#Vector_Ratings_BD=Conn.Conexion_BD_ratings(host, user, passwd, database, table)
#clients_ratings=C_V.Creacion_Matrix_Rating(Vector_Ratings_BD)

class recommenders:
    
    def item_ranking (self, apartment_file_processed,client_file_processed,client_rating,visualization):
        
        #Creacion de SFrames
        
        recommender_matrix=gl.SFrame()
        
        #Asignacion de Arrays
        client_array=client_file_processed["ID_C"]
        client_number=client_array[0]
        zona_array=client_file_processed["Zona"]
        client_zona=zona_array[0]
        
        #Visualizacion por Zona
        if (visualization==0):
            apartment_file_processed=apartment_file_processed[apartment_file_processed["Zona"] == client_zona]
        
        #Creacion de Modelo de Recomendacion por factorizacion
                       
        apartment_fact_recommender= gl.ranking_factorization_recommender.create(client_rating,user_id="ID_C",item_id="ID_V",target="rating",
                                                                            user_data=client_file_processed,
                                                                            item_data=apartment_file_processed,
                                                                            ranking_regularization = 0.1,
                                                                            unobserved_rating_value = 1)   
                                                                            
        
              
        recommender_matrix=apartment_fact_recommender.recommend([client_number])
        recommender_matrix.remove_column('rank')
        
        return recommender_matrix.topk('score', k=20)
    
    def collaborative_ranking (self, apartment_file_processed,clients_file_processed,clients_rating,visualization):
         
        print ("Ingresa la zona donde quieres ver las viviendas que compran personas parecidas a tu perfil?")
        zona=int(input())
        client_number=10
        
        #Creacion de SFrames
        
        recommender_matrix=gl.SFrame()
                    
        #Visualizacion por Zona
        if (visualization==0):
            apartment_file_processed=apartment_file_processed[apartment_file_processed["Zona"] == zona]
        
        #Creacion de Modelo de Recomendacion por Colaboracion
                       
        apartment_collaborative_recommender= gl.item_similarity_recommender.create(clients_rating,user_id="ID_C",item_id="ID_V",target="Rating",
                                                                            user_data=clients_file_processed,
                                                                            item_data=apartment_file_processed)   
                                                                            
        
              
        recommender_matrix=apartment_collaborative_recommender.recommend([client_number])
                                                   
        return recommender_matrix.topk('score', k=20)
    
    def non_personalized_ranking (self, apartment_file_processed,clients_file_processed,clients_rating, client_number):
        
        print ("Ingresa la zona donde quieres ver las viviendas mas populares ?")
        zona=int(input())
                
        #Creacion de SFrames
        
        recommender_matrix=gl.SFrame()
        clients_ratings_zone=gl.SFrame()        
        
        #Visualizacion por Zona
        apartment_file_processed=apartment_file_processed[apartment_file_processed["Zona"] == zona]
        zonas_ID= apartment_file_processed["ID_V"]
                
        #Modificacion de la matriz client
        
        clients_ratings_zone=clients_rating.filter_by(zonas_ID, 'ID_V')
        
              
        #Creacion de Modelo de Recomendacion por Colaboracion
                       
        apartment_popularity_recommender= gl.recommender.popularity_recommender.create(clients_ratings_zone,user_id="ID_C",item_id="ID_V",target="Rating",
                                                                            user_data=clients_file_processed,
                                                                            item_data=apartment_file_processed)   
                                                                            
        
              
        recommender_matrix=apartment_popularity_recommender.recommend([client_number])
        recommender_matrix=recommender_matrix.remove_column("ID_C")
        return recommender_matrix.topk('score', k=20)
    
d=recommenders()
#e=recommenders()
#f=recommenders()
d.item_ranking(apartment_file_processed,client_file_processed,client_rating,visualization)
#e.collaborative_ranking(apartment_file_processed,clients_file_processed,clients_rating,visualization)
#f.non_personalized_ranking(apartment_file_processed,clients_file_processed,clients_rating)

Quieres ver apartamentos fuera de tu zona de escogencia?
1


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.052893 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,long,long,long,long,long,long,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\user\Documents\Sistema de Recomendacion\Single_Client_raw.csv

Parsing completed. Parsed 1 lines in 0.062831 secs.

Recsys training: model = ranking_factorization_recommender

[ERROR] graphlab.toolkits._main: Toolkit error: Target column given as 'rating', but this is not present in the provided data.


ToolkitError: Target column given as 'rating', but this is not present in the provided data.